In [17]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, recall_score, f1_score
from aif360.datasets import BinaryLabelDataset
from aif360.metrics import ClassificationMetric
from sklearn.model_selection import train_test_split
from aif360.algorithms.preprocessing import Reweighing

In [18]:
def compute_performance_metrics(y_test, y_pred, model_name):
    accuracy = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1_score_value = f1_score(y_test, y_pred)
    print(f"{model_name} Accuracy: {accuracy}")
    print(f"{model_name} Recall: {recall}")
    print(f"{model_name} F1 Score: {f1_score_value}")

def compute_classification_metric(dataset,predictions, label_name_v, favorable_label_v, unfavorable_label_v, privileged_attribute, unprivileged_attribute):
    features = [privileged_attribute] + unprivileged_attribute # We want to check the fairness level regarding the protected attribute "sex"

    # This is the object made of the original dataset
    aif_sex_dataset = BinaryLabelDataset( # Base class for all structured datasets with binary labels.
            df=dataset,
            favorable_label=favorable_label_v, # This means that a prediction is biased toward the privileged attribute if its value is 1 (True)
            unfavorable_label=unfavorable_label_v,
            label_names=[label_name_v],
            protected_attribute_names=features,
            privileged_protected_attributes=[privileged_attribute], # here we tell AIF that we want to check for predictions
        )

    # We do the same thing but with the predictions dataset
    aif_sex_pred = BinaryLabelDataset(
            df=predictions,
            favorable_label=favorable_label_v,
            unfavorable_label=unfavorable_label_v,
            label_names=[label_name_v],
            protected_attribute_names=features,
            privileged_protected_attributes=[privileged_attribute],
        )

    sex_privileged_group = [{privileged_attribute: 1, **{attr: 0 for attr in unprivileged_attribute}}]
    sex_unprivileged_group = [{attr: 1, privileged_attribute: 0} for attr in unprivileged_attribute]

    # We provide the ClassificationMetric object with all the information needed:
    # aif_sex_dataset - The original test set
    # aif_sex_pred - A dataset containing the predictions of the model
    # sex_privileged_group - The privileged group
    # sex_unprivileged_group - The unprivileged group
    fairness_metrics = ClassificationMetric(dataset=aif_sex_dataset,
                            classified_dataset=aif_sex_pred,
                            unprivileged_groups=sex_unprivileged_group,
                            privileged_groups=sex_privileged_group)
    
    return fairness_metrics

def compute_fairness_metrics(fairness_metrics: ClassificationMetric):
    # Values less than 0 indicate that privileged group has higher
    # proportion of predicted positive outcomes than unprivileged group.
    # Value higher than 0 indicates that unprivileged group has higher proportion
    # of predicted positive outcomes than privileged group.
    SPD = round(fairness_metrics.statistical_parity_difference(),3)

    # Measures the deviation from the equality of opportunity, which means that the same
    # proportion of each population receives the favorable outcome. This measure must be equal to 0 to be fair.
    EOD = round(fairness_metrics.equal_opportunity_difference(),3)

    # Average of difference in False Positive Rate and True Positive Rate for unprivileged and privileged groups
    # A value of 0 indicates equality of odds, which means that samples in both the privileged and unprivileged
    # groups have the same probability of being classified positively.
    AOD = round(fairness_metrics.average_odds_difference(),3)

    print(f"Statistical Parity Difference (SPD): {SPD}")
    print(f"Average Odds Difference (AOD): {AOD}")
    print(f"Equal Opportunity Difference (EOD): {EOD}")

In [19]:
dataset_path = "datasets/trained_compas-score.csv"
df_raw = pd.read_csv(dataset_path) 

pd.set_option('display.max_columns', None)

In [20]:
X = df_raw.drop(columns="is_recid")
y = df_raw["is_recid"]

In [21]:

sex_features = ['sex_Male','sex_Female']

sex_privileged_group = [{'sex_Male': 0, 'sex_Female': 1}] # The privileged group is made of males (sex_Male = True)
sex_unprivileged_group = [{'sex_Female': 0, 'sex_Male': 1}] # The unprivileged group is made of females (sex_Female = True)

dataset = X.copy(deep=True) # we recreate a copy of the whole dataset processed and balanced
dataset['is_recid'] = y # and join the target feature with the others

# Create a BinaryLabelDataset using AIF360 library
aif_dataset = BinaryLabelDataset(
        df=dataset,
        favorable_label=0,
        unfavorable_label=1,
        label_names=['is_recid'],
        protected_attribute_names=sex_features,
    )

# Apply the Reweighing technique
RW = Reweighing(unprivileged_groups=sex_unprivileged_group, privileged_groups=sex_privileged_group)
dataset_transformed = RW.fit_transform(aif_dataset)

# Get sample weights from the transformed dataset
sample_weights = dataset_transformed.instance_weights

# Create a fair dataset using the reweighing technique
fair_dataset = dataset.copy(deep=True)
fair_dataset['weights'] = sample_weights

# Get features and target from the dataset
features = dataset.columns.tolist()
features.remove('is_recid')
target = ['is_recid']

# Set dataset features and target
X_fair = fair_dataset[features]
y_fair = fair_dataset[target]

X_fair_train, X_fair_test, y_fair_train, y_fair_test, sample_weights_train, sample_weights_test = train_test_split(X_fair, y_fair, sample_weights, test_size=0.2, random_state=42)

In [22]:
dt_clf = DecisionTreeClassifier(random_state=42)

# The fit function will do the trick
dt_clf.fit(X_fair_train, y_fair_train, sample_weight=sample_weights_train)

# After the training phase, the model will be tested by predicting the values on the test set
dt_predictions = dt_clf.predict(X_fair_test)

compute_performance_metrics(y_fair_test,dt_predictions,"Decision Tree")

Decision Tree Accuracy: 0.659706546275395
Decision Tree Recall: 0.46677471636953
Decision Tree F1 Score: 0.48854961832061067


In [23]:
svm_classifier = make_pipeline(StandardScaler(), SVC(kernel='linear'))

# Addestra il classificatore sui dati di training
svm_classifier.fit(X_fair_train, y_fair_train, svc__sample_weight=sample_weights_train)

# Fai delle predizioni sui dati di test
svm_pred = svm_classifier.predict(X_fair_test)

compute_performance_metrics(y_fair_test,svm_pred,"SVM")

c:\Users\mario\.conda\envs\se4faiEnv\lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVM Accuracy: 0.6992099322799097
SVM Recall: 0.2884927066450567
SVM F1 Score: 0.40044994375703036


In [24]:
rf_classifier = RandomForestClassifier(n_estimators=100, criterion='gini', max_depth = None, random_state=42)

rf_classifier.fit(X_fair_train, y_fair_train, sample_weight=sample_weights_train)

rf_predictions = rf_classifier.predict(X_fair_test)

compute_performance_metrics(y_fair_test,rf_predictions,"Random Forest")

c:\Users\mario\.conda\envs\se4faiEnv\lib\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Random Forest Accuracy: 0.7065462753950339
Random Forest Recall: 0.4051863857374392
Random Forest F1 Score: 0.49019607843137253


In [31]:
#CALCOLO LE METRICHE DI FAIRNESS CONSIDERANDO COME MODELLO IL DECISION TREE
dataset = X_fair_test.copy(deep=True) # we create a copy of the test set
dataset['is_recid'] = y_fair_test  # and join the target feature with the others
predictions = dataset.copy(deep=True) # we do the same task
predictions['is_recid'] = dt_predictions # but this time the target feature is made by the predictions of our model

race_privileged = "race_Caucasian"
race_unprivileged = ["race_African-American", "race_Asian", "race_Other", "race_Native American", "race_Hispanic"]
sex_privileged = "sex_Female"
sex_unprivileged = ["sex_Male"]

In [32]:
fairness_metrics = compute_classification_metric(dataset,predictions,"is_recid",0,1,sex_privileged,sex_unprivileged)
compute_fairness_metrics(fairness_metrics)

Statistical Parity Difference (SPD): -0.03
Average Odds Difference (AOD): -0.013
Equal Opportunity Difference (EOD): -0.009


In [33]:
predictions['is_recid'] = svm_pred 

In [34]:
fairness_metrics = compute_classification_metric(dataset,predictions,"is_recid",0,1,sex_privileged,sex_unprivileged)
compute_fairness_metrics(fairness_metrics)

Statistical Parity Difference (SPD): -0.079
Average Odds Difference (AOD): -0.087
Equal Opportunity Difference (EOD): -0.027


In [35]:
predictions['is_recid'] = rf_predictions 

In [36]:
fairness_metrics = compute_classification_metric(dataset,predictions,"is_recid",0,1,sex_privileged,sex_unprivileged)
compute_fairness_metrics(fairness_metrics)

Statistical Parity Difference (SPD): -0.135
Average Odds Difference (AOD): -0.141
Equal Opportunity Difference (EOD): -0.071
